In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Any, Dict, Iterator, List, Mapping, Optional
import requests
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from enum import Enum
import requests
import json

In [2]:
class EmailParser(BaseModel):
    email_sender: str = Field(
        description="The email address of the sender, indicating who sent the email."
    )
    email_receiver: str = Field(
        description="The email address of the recipient, indicating who is intended to receive the email."
    )
    email_body: str = Field(
        description="The main content of the email, which may include text, links, or other information. This field represents the message being conveyed from the sender to the receiver."
    )

# Send request to local ollama server for testing 
def send_request():
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "minicpm-v:8b",
        "prompt": "Why is the sky blue?",
        "stream": False,
        "options": {
            "temperature": 0
        }
    }
    
    headers = {'Content-Type': 'application/json'}
    
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"



# Wrapper of lanchain to call local LLM API
class CustomLLM(LLM):
# https://python.langchain.com/docs/how_to/custom_llm/ 
    url: str
    is_stream: bool
    model: str
    model_path: str
    temperature: float = 0.7

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        print (self.temperature)
        data = {
            "model": self.model,
            "prompt": prompt,
            "stream": self.is_stream,
            "options": {
                "temperature": self.temperature
            }
        }
        
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url=self.url, json=data, headers=headers)
        
        if response.status_code == 200:
            response_data = response.json()
            # Assuming the API response contains the generated text in a field like 'output'
            return response_data.get("response") # Return n characters from the output
        else:
            raise RuntimeError(f"Error {response.status_code}: {response.text}")


    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

In [3]:
email = """

From: <lujun.li@EMC.lu>
To: <Jacob.Alex@foyer.lu>
Dear Jacob,  
  
Except my fault, we didn't receive your reply yet since my last email for my request of reinbursement on the car accident. 
  

Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


-----------

Dear Jacob,  
  
Your opinion matters to us!  
  
You are invited to participate in a short survey about local transportation
issues in the Bay Area.  Your opinions will be used for research purposes only
and your responses will remain completely confidential.  
  
Follow this link to the Survey:  
Take the Survey  
  
Or copy and paste the URL below into your internet browser:  
https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email  
  
Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


Bamboo Spa Grand Reopening Sale!

|  |  
---  
|

4 Options Starting From

$70.00  
  
  
---  
  

View Offer  
  
What We're Offering

**Join in our reopening celebration today by enjoying exclusive offers on
your preferred services!**

Unwind and rejuvenate yourself with our exclusive packages of **Foot
Reflexology** , **Tension Tamer** , and **CustomBlend Massage**.

Make sure to visit us for our inaugural=C2=A0 **Halloween celebration on
October 27th, from 3pm-6pm </strong> (admission is free), where you can
indulge in delightful treats and take advantage of amazing deals **RSVP**
**.** For further details aboutthis month's promotions, kindly refer to our
**=C2=A0** **Specials Page.

**Special Offers for the Grand Reopening!**

****__* 1 hr Foot Reflexology $70__

____* 30 min Tension Tamer Massage + 30 min Foot Reflexology$85__

____* 60 min Custom Blend Massage $105__

____* 90 min Custom Blend Massage $125__

**Foot Reflexology:=C2=A0** Revitalizing and refreshing the feet and legs is
accomplished by stimulating key pressure points either manually or using a
reflexology stick, through foot reflexology.

**Tension Tamer:=C2=A0** Ease tension in your neck or unwindwith a soothing
scalp massage. Upon request, enjoy essential oils or lotion; otherwise
receive a dry treatment.

**Custom Blend Massage:** =C2=A0Our expert therapists with awealth of
experience will skillfully combine various techniques to relievetension,
promote relaxation and provide relief for your muscles.

For questions or more information, please call 415-567-8812 or visit our=
website at http://www.bamboospasf.com.  
  
|  

Bamboo Spa SF

415-567-8812

www.bamboospasf.com

frontdesk@bamboospasf.com

|

|

2284 Union St  
San Francisco, CA 94123  
  
---  
  
|  Find more local experiences and promotions  
|

  

|  
  
---|---  
This email was created by Hownd on behalf of Bamboo Spa SF  
(Located at 2284 Union St, San Francisco, CA, 94123).  
  
  
Please review our Privacy Policy.  
Unsubscribe to stop receiving this email.  
Manage Privacy
"""

criteria = """
1. The sender are waiting for a response from the receiver.
2. The email is a reminder of a past request.
3. The email is a reminder of a reminder."""


The color of the sky appears to be a shade of blue because it reflects sunlight. The Earth's atmosphere scatters light in all directions, but short-wavelength visible light (blue and violet) are scattered more than others due to their shorter wavelengths. This scattering effect causes most of this light to scatter into our eyes from every direction we look at the sky during daylight hours.

The reason why blue is a dominant color for sunlight that reaches us on Earth's surface, as opposed to other colors like red or green, has to do with how much each wavelength scatters in different directions. Blue and violet wavelengths are scattered more than others because they have shorter wavelengths which means their particles move faster through the air molecules before hitting another one.

The scattering of light is also why we see stars at night as bright points instead of diffuse disks like during daytime when all visible light from a star has been scattered in every direction by Earth's a

In [6]:
# Initialize local LLM model for calling

llm = CustomLLM(url = "http://localhost:11434/api/generate",is_stream = False,  model = "minicpm-v:8b", model_path="minicpm-v:8b")

In [8]:
# Define the Chain
from pydantic import ValidationError
parser = JsonOutputParser(pydantic_object=EmailParser)

prompt = PromptTemplate(
    template="Can you provide details or insights on the very first email found at the top of the document below? \nMake sure to answer in the following format.{format_instructions}.\nHere is the document:\n{input}",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

llm.temperature = 0.7
def invoke_chain_with_retry(input_data, max_retries=10):
    attempts = 0
    while attempts < max_retries:
        response = chain.invoke({"input": input_data})
        return response  # Successfully parsed output

# Call the function with the email input
result = invoke_chain_with_retry(email)

0.7


In [9]:
result

{'email_sender': 'lujun.li@EMC.lu',
 'email_receiver': 'Jacob.Alex@foyer.lu',
 'email_body': "Dear Jacob,\n\nExcept my fault, we didn't receive your reply yet since my last email for my request of reinbursement on the car accident.\n\nThank you for your participation!\n\nEMC Research is a national market and opinion research firm that conducts research on a variety of topics. We are not trying to sell anything, and your responses will remain completely anonymous. To learn more about EMC Research,\ngo to www.emcresearch.com.\n\nFollow the link to opt out of future emails: Click here to unsubscribe",
 'email_type': 'reminder'}

In [25]:
class EmailClassification(BaseModel):
    is_reminder: bool = Field(
        description="Whether the email document is classified as a reminder email."
    )
    justification: str = Field(
        description="Explanation of why the email is classified as a reminder."
    )


parser = JsonOutputParser(pydantic_object=EmailClassification)
prompt = PromptTemplate(
    template="Can you please tell classfify whether the document is a 'reminder' according to one of the following the criteria? {criteria} \nMake sure to answer in the following format.{format_instructions}.\nHere is the document:\n{input}",
    input_variables=["criteria", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llm | parser

llm.temperature = 0.7
def invoke_chain_with_retry(max_retries=10):
    attempts = 0
    while attempts < max_retries:
        response = chain.invoke({"criteria": criteria, "input": result["email_body"]})
        return response  # Successfully parsed output

# Call the function with the email input
output = invoke_chain_with_retry()


0.7


In [28]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

examples = [
            {"question": "dfasdf","answer": """

             """}, 
            {"question": "dfasdf","answer": """

             """}]

print(example_prompt.invoke(examples[0]).to_string())

Question: dfasdf
"

             


In [29]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: dfasdf
"

             

Question: dfasdf
"

             

Question: Who was the father of Mary Ball Washington?
